# 目的
AkaNeでは配信していたクリエイティブが、ReeMoでは却下になっているケースがある。

AkaNeとReeMoで同じクリエイティブがあるかを判断したい。

テキストは空白を削除して一致するかどうかで判断できるが、画像は判断ができないので、

簡単な類似度の計算で一致率を算出し、それをもとに対応表を作りたい。

# 想定している設計
1. AkaNeで配信していたクリエイティブのURLを、BigQueryから取得する。
2. 大画像(横長の画像)と小画像(正方形の画像)で分けで、取得したURLから画像をダウンロードする。
3. ReeMoでも1. と2. を行う
4. ReeMoの画像サイズがAkaNeと異なる可能性があるため、ReeMoの画像をAkaNeの画像サイズにそろえる(縦横比は変えない)
5. AkaNeの小画像とReeMoの小画像で、類似度を計算する。類似度の計算方法は下記のリンクを参考にする。画素値が良さそう。
6. もっとも類似度が高い組み合わせを残し、次のデータフレームを作る
|AkaNeのcreative_id|AkaNeの画像URL|ReeMoのcreative_id|ReeMoの画像URL|類似度|
7. 5と6の処理を大画像でも行う。
8. できたデータフレームをBigQueryにアップロードする。

https://qiita.com/jun_higuche/items/752ef756a182261fcc55

#AkaNeの原稿の画像のダウンロードする
「N連/TAXEL枠で同じようなクリエイティブを被らせない」に対応するため、機械学習モデルに使用する教師データに利用する目的でダウンロードする。

https://trello.com/c/6x7HZJpU

BigQueryに画像URLが入っているので、そこからURLリストを取得するためにBQと接続する。

In [ ]:
from google.colab import auth
auth.authenticate_user()

必要なライブラリをインポート

In [ ]:
!pip install imagehash

In [ ]:
import pandas as pd
import os
import pprint
import time
import urllib.error
import urllib.request
import shutil
import hashlib
from glob import glob

Colab上でSQLを実行し、画像URLをDataFrameで取得する。

ラベル付けできない「その他」の画像が多くなってしまうため、取得する画像の業種を美容アフィ系の業種に絞った。

In [ ]:
%%bigquery --project gmo-am-pmp image_url_df
#StandardSQL
#StandardSQL
CREATE TEMPORARY FUNCTION START_DATE() AS ("20230929");
CREATE TEMPORARY FUNCTION END_DATE() AS ("20230929");
WITH
  AKANE_REPORT AS
  (
  SELECT
    ad_id,
    sponsor_id,
    campaign_id,
    SUM(gross_sum) AS gross
  FROM
    `gmo-am-pmp.adcloud_analysis.report_master_*`
  WHERE
    _TABLE_SUFFIX BETWEEN START_DATE() AND END_DATE()
    AND ad_id != 0
    AND network_id = 211
  GROUP BY
    ad_id,
    sponsor_id,
    campaign_id
    ),
  CREATIVE AS
  (
  SELECT
    id,
    title,
    text,
    destination,
    alter_destination,
    sponsor_id
  FROM
    `gmo-am-pmp.adcloud_analysis.creatives_master_*`
  WHERE
    _TABLE_SUFFIX BETWEEN START_DATE() AND END_DATE()
  GROUP BY
    id,
    title,
    text,
    destination,
    alter_destination,
    sponsor_id
    ),
  SPONSOR AS
  (
  SELECT
    id,
    company_name
  FROM
    `gmo-am-pmp.adcloud_analysis.sponsor_master_*`
  WHERE
    _TABLE_SUFFIX = END_DATE()
  GROUP BY
    id,
    company_name
   ),
  CAMPAGIN AS
  (
  SELECT
    id,
    name,
    sector_id
  FROM
    `gmo-am-pmp.adcloud_analysis.campaign_master_*`
  WHERE
    _TABLE_SUFFIX = END_DATE()
  GROUP BY
    id,
    name,
    sector_id
    ),
  ADS_MASTER AS
  (
  SELECT
    id,
    creative_id
  FROM
    `gmo-am-pmp.adcloud_analysis.ads_master_*`
  WHERE
    _TABLE_SUFFIX = END_DATE()
   GROUP BY
    id,
    creative_id
    )
SELECT
  AKANE_REPORT.ad_id,
  AKANE_REPORT.sponsor_id,
  SPONSOR.company_name,
  AKANE_REPORT.campaign_id,
  CAMPAGIN.name,
  CREATIVE.title,
  CREATIVE.text,
  CONCAT("https://adn-i.sp.gmossp-sp.jp", CREATIVE.destination) AS large_image,
  CONCAT("https://adn-i.sp.gmossp-sp.jp", CREATIVE.alter_destination) AS small_image
FROM
  AKANE_REPORT
  INNER JOIN
    ADS_MASTER
    ON
      AKANE_REPORT.ad_id = ADS_MASTER.id
      INNER JOIN
        CREATIVE
          ON
            ADS_MASTER.creative_id = CREATIVE.id
            INNER JOIN
              SPONSOR
                ON
                  AKANE_REPORT.sponsor_id = SPONSOR.id
                  INNER JOIN
                    CAMPAGIN
                      ON
                        AKANE_REPORT.campaign_id = CAMPAGIN.id
GROUP BY
  AKANE_REPORT.ad_id,
  AKANE_REPORT.sponsor_id,
  SPONSOR.company_name,
  AKANE_REPORT.campaign_id,
  CAMPAGIN.name,
  CREATIVE.title,
  CREATIVE.text,
  large_image,
  small_image
ORDER BY
  AKANE_REPORT.sponsor_id ASC
;

取得したデータを確認

In [ ]:
image_url_df.head()

,ad_id,sponsor_id,company_name,campaign_id,name,title,text,large_image,small_image
0,4155539,6892,株式会社ナチュラルプランツ,77226,[カテゴリB]【リーチ】インフィード配信_libido,気になる人以外してはいけない,100人中92人の男性が「Hしたい」とスイッチが入るのは女性の…,None,https://adn-i.sp.gmossp-sp.jp/creatives/2e5/ae...
1,4338500,6892,株式会社ナチュラルプランツ,42306,[カテゴリB]★【運用】インフィード配信_リターゲティング,もっと私に夢中になって欲しい！,涙が出るほど気持ちいいHって？男性が溺愛する夜のテクニック傑作3選,None,https://adn-i.sp.gmossp-sp.jp/creatives/2c7/67...
2,5123945,6892,株式会社ナチュラルプランツ,45552,[カテゴリB]【リーチ】インフィード配信_ブロード_CPM配信,もっと早く知りたかった…,男性100人中92人が「Hしたくなる」興奮スイッチが入るのは女性の…,None,https://adn-i.sp.gmossp-sp.jp/creatives/fe9/fd...
3,5183288,6892,株式会社ナチュラルプランツ,103592,[カテゴリB]【リーチ】インフィード配信_libido_クリエイティブテスト,もっと早く知りたかった…,男性100人中92人が「Hしたくなる」興奮スイッチが入るのは女性の…,None,https://adn-i.sp.gmossp-sp.jp/creatives/0d0/46...
4,5183300,6892,株式会社ナチュラルプランツ,103595,[カテゴリB]【リーチ】インフィード配信_kiji02_クリエイティブテスト,もっと私に夢中になって欲しい！,涙が出るほど気持ちいいHって？男性が溺愛する夜のテクニック傑作4選,None,https://adn-i.sp.gmossp-sp.jp/creatives/f4b/30...


In [ ]:
len(image_url_df.index)

2764

画像名のカラムを作成

In [ ]:
image_url_df['image_name'] = image_url_df['small_image'].replace('.*/','',regex=True)

データを確認

In [ ]:
image_url_df.head()

,ad_id,sponsor_id,company_name,campaign_id,name,title,text,large_image,small_image,image_name
0,4155539,6892,株式会社ナチュラルプランツ,77226,[カテゴリB]【リーチ】インフィード配信_libido,気になる人以外してはいけない,100人中92人の男性が「Hしたい」とスイッチが入るのは女性の…,None,https://adn-i.sp.gmossp-sp.jp/creatives/2e5/ae...,09b95715c68979a71627984395599672_original.jpg
1,4338500,6892,株式会社ナチュラルプランツ,42306,[カテゴリB]★【運用】インフィード配信_リターゲティング,もっと私に夢中になって欲しい！,涙が出るほど気持ちいいHって？男性が溺愛する夜のテクニック傑作3選,None,https://adn-i.sp.gmossp-sp.jp/creatives/2c7/67...,b17243fe30c132d61631260917701746_original.jpg
2,5123945,6892,株式会社ナチュラルプランツ,45552,[カテゴリB]【リーチ】インフィード配信_ブロード_CPM配信,もっと早く知りたかった…,男性100人中92人が「Hしたくなる」興奮スイッチが入るのは女性の…,None,https://adn-i.sp.gmossp-sp.jp/creatives/fe9/fd...,d7fbd150ef97dcdb16466205538062036_original.jpg
3,5183288,6892,株式会社ナチュラルプランツ,103592,[カテゴリB]【リーチ】インフィード配信_libido_クリエイティブテスト,もっと早く知りたかった…,男性100人中92人が「Hしたくなる」興奮スイッチが入るのは女性の…,None,https://adn-i.sp.gmossp-sp.jp/creatives/0d0/46...,9147b8f29fd2c78a16480934986748006_original.jpg
4,5183300,6892,株式会社ナチュラルプランツ,103595,[カテゴリB]【リーチ】インフィード配信_kiji02_クリエイティブテスト,もっと私に夢中になって欲しい！,涙が出るほど気持ちいいHって？男性が溺愛する夜のテクニック傑作4選,None,https://adn-i.sp.gmossp-sp.jp/creatives/f4b/30...,28cd2ceace7b59ef16480939326289372_original.jpg


URLをlistで取得

In [ ]:
small_df = image_url_df.dropna(subset=['small_image'])
url_list = small_df['small_image'].values.tolist()
print(url_list)
print(len(url_list))

打増をダウンロードする関数を定義

In [ ]:
def download_file(url, dst_path):
    try:
        with urllib.request.urlopen(url) as web_file, open(dst_path, 'wb') as local_file:
            local_file.write(web_file.read())
    except urllib.error.URLError as e:
        print(e)

def download_file_to_dir(url, dst_dir):
    download_file(url, os.path.join(dst_dir, os.path.basename(url)))

ダウンロード先のディレクトリを指定

In [ ]:
download_dir = '/content/train_images'

指定したディレクトリにフォルダを作成

In [ ]:
try:
    shutil.rmtree(download_dir)
except FileNotFoundError:
    pass

#同名のフォルダを作成する
os.mkdir(download_dir)

URLのリストから画像をダウンロード

In [ ]:
sleep_time_sec = 0.5

for i,url in enumerate(url_list):
    print(url,end=', ')
    print(i+1,end='/')
    print(len(url_list))
    download_file_to_dir(url, download_dir)
    time.sleep(sleep_time_sec)

ダウンロードした中で、まったく同じ画像を削除(同じ画像にラベルを付ける手間の削減&学習データに偏りを生まないため)

In [ ]:
flist = []
fmd5 = []
dl = []

dirname = '/content/train_images/*'

flist.extend(glob('/content/train_images/*'))

for fn in flist:
  with open(fn, 'rb') as fin:
    data = fin.read()
    m = hashlib.md5(data)
    fmd5.append(m.hexdigest())

for i in range(len(flist)):
  if flist[i] in dl: continue
  for j in range(i+1, len(flist)):
    if flist[j] in dl: continue
    if fmd5[i] == fmd5[j] and not flist[j] in dl:
      dl.append(flist[j])

for a in dl: os.remove(a)
print(dl)

ダウンロードした画像をzipに圧縮してローカルに保存(モデルの再学習に使用する)

In [ ]:
# ダウンロードしたいフォルダを zip 圧縮する
!zip -r /content/download.zip /content/train_images

# 圧縮した zip ファイルをダウンロードする
from google.colab import files
files.download("/content/download.zip")

画像URLとad_idの対応表をCSVで保存(画像に対応するラベルを与えるのに使用する)

In [ ]:
small_df[['ad_id','sponsor_id','campaign_id','small_image','image_name']].to_csv('/content/small_url.csv',encoding='utf_8_sig')

学習データには使用しないが、テスト用に大画像でも同じことした。

In [ ]:
image_url_df['large_image_name'] = image_url_df['large_image'].replace('.*/','',regex=True)

In [ ]:
large_df = image_url_df.dropna(subset=['large_image'])
url_list = large_df['large_image'].values.tolist()
print(url_list)
print(len(url_list))

In [ ]:
def download_file(url, dst_path):
    try:
        with urllib.request.urlopen(url) as web_file, open(dst_path, 'wb') as local_file:
            local_file.write(web_file.read())
    except urllib.error.URLError as e:
        print(e)

def download_file_to_dir(url, dst_dir):
    download_file(url, os.path.join(dst_dir, os.path.basename(url)))

In [ ]:
sleep_time_sec = 0.5

for i,url in enumerate(url_list):
    print(url,end=', ')
    print(i+1,end='/')
    print(len(url_list))
    download_file_to_dir(url, download_dir)
    time.sleep(sleep_time_sec)

In [ ]:
flist = []
fmd5 = []
dl = []

dirname = '/content/train_images/*'

flist.extend(glob('/content/train_images/*'))

for fn in flist:
  with open(fn, 'rb') as fin:
    data = fin.read()
    m = hashlib.md5(data)
    fmd5.append(m.hexdigest())

for i in range(len(flist)):
  if flist[i] in dl: continue
  for j in range(i+1, len(flist)):
    if flist[j] in dl: continue
    if fmd5[i] == fmd5[j] and not flist[j] in dl:
      dl.append(flist[j])

for a in dl: os.remove(a)
print(dl)

In [ ]:
large_df[['ad_id','sponsor_id','campaign_id','large_image','image_name']].to_csv('/content/large_url.csv',encoding='utf_8_sig')

In [ ]:
# ダウンロードしたいフォルダを zip 圧縮する
!zip -r /content/download.zip /content/train_images

# 圧縮した zip ファイルをダウンロードする
from google.colab import files
files.download("/content/download.zip")

# ReeMoの画像をダウンロードする

In [ ]:
%%bigquery --project gmo-am-pmp image_url_df
with
report AS (
SELECT
  creative_id,
  SUM(amount) AS gross
FROM
  `reemo-173606.reemo_report.report_creative_dailies_*`
where _table_suffix = format_date("%Y%m%d", date_sub(current_date('Asia/Tokyo'),interval 0 day))
GROUP BY
  creative_id
),
creatives as(
select
  --advertiser_id,
  id,
  creative_group_id,
  creative_id,
  image_id,
  image,
  flg,
  --small_image_id,
  --small_image,
  --large_image_id,
  --large_image,
  title_id,
  title,
  text_id,
  text
from
  `reemo-173606.reemo_dump.natives_2*`
  left join (select native_id id,large_image_id image_id,1 flg from `reemo-173606.reemo_dump.natives_large_images_*` where _table_suffix = format_date("%Y%m%d", date_sub(current_date('Asia/Tokyo'),interval 0 day)) union all select native_id id,small_image_id image_id, 0 flg from `reemo-173606.reemo_dump.natives_small_images_*` where _table_suffix = format_date("%Y%m%d", date_sub(current_date('Asia/Tokyo'),interval 0 day))) using(id)
  --left join (select native_id id,small_image_id from `reemo-173606.reemo_dump.natives_small_images_20230413`) using(id)
  left join (select native_id id,text_id from `reemo-173606.reemo_dump.natives_texts_*` where _table_suffix = format_date("%Y%m%d", date_sub(current_date('Asia/Tokyo'),interval 0 day))) using(id)
  left join (select native_id id,title_id from `reemo-173606.reemo_dump.natives_titles_*` where _table_suffix = format_date("%Y%m%d", date_sub(current_date('Asia/Tokyo'),interval 0 day))) using(id)
  left join (select id text_id,text,advertiser_id from `reemo-173606.reemo_dump.texts_*` where _table_suffix = format_date("%Y%m%d", date_sub(current_date('Asia/Tokyo'),interval 0 day))) using(text_id)
  left join (select id title_id,title from `reemo-173606.reemo_dump.titles_*` where _table_suffix = format_date("%Y%m%d", date_sub(current_date('Asia/Tokyo'),interval 0 day))) using(title_id)
  left join (select id image_id,concat("https://image.dsp.reemo-ad.jp/",image_store_path) image from `reemo-173606.reemo_dump.small_images_*` where _table_suffix = format_date("%Y%m%d", date_sub(current_date('Asia/Tokyo'),interval 0 day)) union all select id image_id,concat("https://image.dsp.reemo-ad.jp/",image_store_path) image from `reemo-173606.reemo_dump.large_images_*` where _table_suffix = format_date("%Y%m%d", date_sub(current_date('Asia/Tokyo'),interval 0 day))) using(image_id)
  --left join (select id large_image_id,concat("https://image.dsp.reemo-ad.jp/",image_store_path) large_image from `reemo-173606.reemo_dump.large_images_20230413`) using(large_image_id)
where
  _table_suffix = right(format_date("%Y%m%d", date_sub(current_date('Asia/Tokyo'),interval 0 day)) ,7)
),
cr as(
select
  id creative_id,
  advertiser_id,
  ad_group_id,
  creative_group_id,
  creative_group_name,
  creatable_type,
  state reemo_state
from
  `reemo-173606.reemo_dump.creatives_2023*`
  left join (
    select
      ad_group_id,
      creative_group_id
    from
      `reemo-173606.reemo_dump.ad_groups_creative_groups_*`
    where
      _table_suffix = format_date("%Y%m%d",current_date('Asia/Tokyo'))
  ) using(creative_group_id)
  left join(
    select
      id creative_group_id,
      name creative_group_name
    from
      `reemo-173606.reemo_dump.creative_groups_*`
    where
      _table_suffix = format_date("%Y%m%d",current_date('Asia/Tokyo'))
  ) using(creative_group_id)
where
  _table_suffix = right(format_date("%Y%m%d",current_date('Asia/Tokyo')),4)
),
exam as(
select
  creative_id,
  slot_id,
  case
    when result = 1 then "◯"
    when result = 2 then "×"else null end ssp_result,
  hour,
  rk
from(
select
  creative_id,
  slot_id,
  result,
  hour,
  rank() over(partition by slot_id,creative_id,result order by hour desc) rk
from
  `reemo-173606.reemo_report.creative_gmossp_judged_*`
where
  _table_suffix = format_date("%Y%m%d",current_date('Asia/Tokyo'))
)
where
  rk = 1
)
select
  concat(advertiser_id,":",advertiser_name) advertiser_id_name,
  concat(ad_group_id,":", ad_group_name)ad_group_id_name,
  concat(creative_group_id,":", creative_group_name) creative_group_id_name,
  creative_id,
  creatable_type,
  flg,
  image,
  title,
  text,
  reemo_state,
  gross
from
  report
left join
  cr
using(creative_id)
  left join `reemo-173606.reemo_log.reemo_advertiser_master` using(advertiser_id,ad_group_id)
  left join creatives using(creative_group_id,creative_id)
order by
  gross DESC
;

# ローカルからダウンロード


In [ ]:
import pandas as pd

df = pd.read_csv("/content/ReeMo画像URLv1.1.csv")

In [ ]:
sample_reemo_df = df.query('ad_group_id == 19406 and flg == 0.0')
sample_reemo_df.head()

,advertiser_id,advertiser_name,ad_group_id,ad_group_name,creative_group_id_name,creative_id,creatable_type,flg,image,title,text,reemo_state,gross
30,5972,グロリアス製薬株式会社(クレブラックリムーバー),19406,◎ブロード配信_【02】新イラスト漫画記事,44720:19406_◎ブロード配信_新基準_【02】新イラスト漫画記事,2626853,Native,0.0,https://image.dsp.reemo-ad.jp/uploads/small_im...,VIOもツルツルにできる裏ワザ,「ムダ毛剃るよりコレ塗って」VIOもツルツルにできるおうち脱毛の裏技,1,7982.462834
58,5972,グロリアス製薬株式会社(クレブラックリムーバー),19406,◎ブロード配信_【02】新イラスト漫画記事,44720:19406_◎ブロード配信_新基準_【02】新イラスト漫画記事,2696309,Native,0.0,https://image.dsp.reemo-ad.jp/uploads/large_im...,VIOもツルツルにできる裏ワザ,『ムダ毛剃るのやめる人続出』9割が知らない！VIOもツルツルにできる裏技,1,5674.930873
59,5972,グロリアス製薬株式会社(クレブラックリムーバー),19406,◎ブロード配信_【02】新イラスト漫画記事,44720:19406_◎ブロード配信_新基準_【02】新イラスト漫画記事,2696309,Native,0.0,https://image.dsp.reemo-ad.jp/uploads/small_im...,VIOもツルツルにできる裏ワザ,『ムダ毛剃るのやめる人続出』9割が知らない！VIOもツルツルにできる裏技,1,5674.930873
114,5972,グロリアス製薬株式会社(クレブラックリムーバー),19406,◎ブロード配信_【02】新イラスト漫画記事,44720:19406_◎ブロード配信_新基準_【02】新イラスト漫画記事,2628104,Native,0.0,https://image.dsp.reemo-ad.jp/uploads/large_im...,VIOもツルツルにできる裏ワザ,『脱毛行くのやめる人続出』9割が知らない！アソコもツルツルにできる裏技,1,3686.398838
115,5972,グロリアス製薬株式会社(クレブラックリムーバー),19406,◎ブロード配信_【02】新イラスト漫画記事,44720:19406_◎ブロード配信_新基準_【02】新イラスト漫画記事,2628104,Native,0.0,https://image.dsp.reemo-ad.jp/uploads/small_im...,VIOもツルツルにできる裏ワザ,『脱毛行くのやめる人続出』9割が知らない！アソコもツルツルにできる裏技,1,3686.398838


In [ ]:
url_list = sample_reemo_df['image'].values.tolist()
print(url_list)
print(len(url_list))

['https://image.dsp.reemo-ad.jp/uploads/small_image/000/094/518/c2cdf7a78.jpg', 'https://image.dsp.reemo-ad.jp/uploads/large_image/000/075/881/c945e8899.jpg', 'https://image.dsp.reemo-ad.jp/uploads/small_image/000/075/881/c3a5ae8e7.jpg', 'https://image.dsp.reemo-ad.jp/uploads/large_image/000/075/884/c39320132.jpg', 'https://image.dsp.reemo-ad.jp/uploads/small_image/000/075/884/c2a2b3b62.jpg', 'https://image.dsp.reemo-ad.jp/uploads/small_image/000/084/152/ca6c36df8.jpg', 'https://image.dsp.reemo-ad.jp/uploads/large_image/000/075/878/ca0f0946d.jpg', 'https://image.dsp.reemo-ad.jp/uploads/small_image/000/075/878/cb1a40a3a.jpg', 'https://image.dsp.reemo-ad.jp/uploads/small_image/000/097/352/c9509f6dc.jpg']
9


In [ ]:
def download_file(url, dst_path):
    try:
        with urllib.request.urlopen(url) as web_file, open(dst_path, 'wb') as local_file:
            local_file.write(web_file.read())
    except urllib.error.URLError as e:
        print(e)

def download_file_to_dir(url, dst_dir):
    download_file(url, os.path.join(dst_dir, os.path.basename(url)))

In [ ]:
download_dir = '/content/reemo_sample_images'

In [ ]:
sleep_time_sec = 0.5

for i,url in enumerate(url_list):
    print(url,end=', ')
    print(i+1,end='/')
    print(len(url_list))
    download_file_to_dir(url, download_dir)
    time.sleep(sleep_time_sec)

https://image.dsp.reemo-ad.jp/uploads/small_image/000/094/518/c2cdf7a78.jpg, 1/9
https://image.dsp.reemo-ad.jp/uploads/large_image/000/075/881/c945e8899.jpg, 2/9
https://image.dsp.reemo-ad.jp/uploads/small_image/000/075/881/c3a5ae8e7.jpg, 3/9
https://image.dsp.reemo-ad.jp/uploads/large_image/000/075/884/c39320132.jpg, 4/9
https://image.dsp.reemo-ad.jp/uploads/small_image/000/075/884/c2a2b3b62.jpg, 5/9
https://image.dsp.reemo-ad.jp/uploads/small_image/000/084/152/ca6c36df8.jpg, 6/9
https://image.dsp.reemo-ad.jp/uploads/large_image/000/075/878/ca0f0946d.jpg, 7/9
https://image.dsp.reemo-ad.jp/uploads/small_image/000/075/878/cb1a40a3a.jpg, 8/9
https://image.dsp.reemo-ad.jp/uploads/small_image/000/097/352/c9509f6dc.jpg, 9/9


In [ ]:
akane_df = pd.read_csv("/content/AkaNe画像URL.csv")
akane_small_df = akane_df.dropna(subset=['small_image'])
akane_small_df.head()


,ad_id,sponsor_id,company_name,campaign_id,name,title,text,large_image,small_image
1,6411953,5636,株式会社ミアキス(神言鑑定),144725,コピー 【枠指定・再開不可】ブロード_インフィード配信(幸せのレシピ)_5月好調枠,運命の人、知りたい？無料占い,運命の人知りたい？名前も顔もズバリ当てるわよ／無料占い,NaN,https://adn-i.sp.gmossp-sp.jp/creatives/f8c/1b...
2,6338798,5636,株式会社ミアキス(神言鑑定),144725,コピー 【枠指定・再開不可】ブロード_インフィード配信(幸せのレシピ)_5月好調枠,誕生日で運命は決まっている！？,「運命の人」に出会いたいなら試すべき占い！ゾッとするほど当たります…,NaN,https://adn-i.sp.gmossp-sp.jp/creatives/d71/e7...
3,6338795,5636,株式会社ミアキス(神言鑑定),144725,コピー 【枠指定・再開不可】ブロード_インフィード配信(幸せのレシピ)_5月好調枠,誕生日で運命は決まっている！？,的中率97％！あなたの将来の結婚相手は生年月日でわかります／無料占い,NaN,https://adn-i.sp.gmossp-sp.jp/creatives/15b/a1...
10,5668142,10427,株式会社FORDELソリューションズ（meemo）,134918,※新【代理店運用】イラスト×漫画記事用[カテゴリD]ブロード_インフィード配信（PC、SP）...,胸だけムチムチにする裏ワザ大公開,貧乳女性の9割知らない：おっぱいだけムチムチにできる裏ワザ大公開,NaN,https://adn-i.sp.gmossp-sp.jp/creatives/c4a/2e...
12,6377234,10427,株式会社FORDELソリューションズ（meemo）,136772,※新【代理店運用】【iOS】[カテゴリD]ブロード_インフィード配信（PC、SP）_新基準,胸だけムチムチにする裏ワザ大公開,豆乳超え「胸パツパツ習慣」閉経貧乳ドン底女がFカップで激モテ,NaN,https://adn-i.sp.gmossp-sp.jp/creatives/e63/bf...


In [ ]:
sample_akane_df = akane_small_df.query('campaign_id == 132128')
sample_akane_df.head()

,ad_id,sponsor_id,company_name,campaign_id,name,title,text,large_image,small_image
524,6049157,19187,グロリアス製薬株式会社(クレブラックリムーバー),132128,【代理店運用】[カテゴリD]ブロード_インフィード配信（PC、SP）_新基準,VIOもツルツルにできる裏ワザ,「ムダ毛剃るよりコレ塗って」VIOもツルツルにできる簡単5分脱毛の裏技,NaN,https://adn-i.sp.gmossp-sp.jp/creatives/637/d7...
532,6084923,19187,グロリアス製薬株式会社(クレブラックリムーバー),132128,【代理店運用】[カテゴリD]ブロード_インフィード配信（PC、SP）_新基準,VIOもツルツルにできる裏ワザ,「ムダ毛剃るよりコレ塗って」VIOもツルツルにできるおうち脱毛の裏技,NaN,https://adn-i.sp.gmossp-sp.jp/creatives/07a/d2...
546,6448289,19187,グロリアス製薬株式会社(クレブラックリムーバー),132128,【代理店運用】[カテゴリD]ブロード_インフィード配信（PC、SP）_新基準,『ムダ毛剃るのやめる人続出』,『ムダ毛剃るのやめる人続出』毛深いVIOも○〇塗るだけ！自宅で5分のツルツル裏技,NaN,https://adn-i.sp.gmossp-sp.jp/creatives/012/72...
549,6258425,19187,グロリアス製薬株式会社(クレブラックリムーバー),132128,【代理店運用】[カテゴリD]ブロード_インフィード配信（PC、SP）_新基準,VIOもツルツルにできる裏ワザ,「ムダ毛剃るよりコレ塗って」VIOもツルツルにできるおうち脱毛の裏技,NaN,https://adn-i.sp.gmossp-sp.jp/creatives/fcc/5c...


In [ ]:
url_list = sample_akane_df['small_image'].values.tolist()
print(url_list)
print(len(url_list))

['https://adn-i.sp.gmossp-sp.jp/creatives/637/d78/67d/ccd36c3e96aad1021675416046892181_original.jpg', 'https://adn-i.sp.gmossp-sp.jp/creatives/07a/d27/0f3/60054103c670db9b1677561555761117_original.jpg', 'https://adn-i.sp.gmossp-sp.jp/creatives/012/727/2b4/17b1ea55ec3ff6e816908812243319352_original.jpg', 'https://adn-i.sp.gmossp-sp.jp/creatives/fcc/5c4/137/2b51e87d08e2896e16841412525377045_original.jpg']
4


In [ ]:
download_dir = "/content/Akane_sample_images_small"

In [ ]:
sleep_time_sec = 0.5

for i,url in enumerate(url_list):
    print(url,end=', ')
    print(i+1,end='/')
    print(len(url_list))
    download_file_to_dir(url, download_dir)
    time.sleep(sleep_time_sec)

https://adn-i.sp.gmossp-sp.jp/creatives/637/d78/67d/ccd36c3e96aad1021675416046892181_original.jpg, 1/4
https://adn-i.sp.gmossp-sp.jp/creatives/07a/d27/0f3/60054103c670db9b1677561555761117_original.jpg, 2/4
https://adn-i.sp.gmossp-sp.jp/creatives/012/727/2b4/17b1ea55ec3ff6e816908812243319352_original.jpg, 3/4
https://adn-i.sp.gmossp-sp.jp/creatives/fcc/5c4/137/2b51e87d08e2896e16841412525377045_original.jpg, 4/4


# ReeMo 小画像とAkane 小画像の比較

In [ ]:
"""
２画像のヒストグラム比較による類似度の算出
"""
import cv2, os

akane_dirname =  '/content/Akane_sample_images_small'
reemo_dirname = '/content/reemo_sample_images'

#akane_cv_dirname = os.path.dirname(akane_dirname)
reemo_cv_dirname = os.path.dirname(reemo_dirname)

akane_files_file = [
    f for f in os.listdir(akane_dirname) if os.path.isfile(os.path.join(akane_dirname, f))
]

reemo_files_file = [
    f for f in os.listdir(reemo_dirname) if os.path.isfile(os.path.join(reemo_dirname, f))
]
similarity_df = []

for i in akane_files_file:
  akane_image = cv2.imread(akane_dirname +'/' + i)

#image1 = imread(akane_dirname + '\\1_1.png')
#image2 = imread(dirname + '\\1_2.png')
#image3 = imread(dirname + '\\2.png')
#image4 = imread(dirname + '\\3.png')

  height = akane_image.shape[0]
  width = akane_image.shape[1]

  img_size = (int(width), int(height))

  for r in reemo_files_file:
    reemo_image = cv2.imread(reemo_dirname + '/' + r)

# 比較するために、同じサイズにリサイズしておく
    image = cv2.resize(reemo_image, img_size)
    print(i + "と"  + r + "の類似度：" + str(np.count_nonzero(akane_image == image) / image.size))
    if np.count_nonzero(akane_image == image) / image.size >= 0.99:
      similarity_df.append((i, r, np.count_nonzero(akane_image == image) / image.size))


similarity_df = pd.DataFrame(similarity_df)

In [ ]:
similarity_df= similarity_df.set_axis(['Akane_image', 'ReeMo_image', 'Similarity'], axis=1)
similarity_df.head()

,Akane_image,ReeMo_image,Similarity
0,ccd36c3e96aad1021675416046892181_original.jpg,ca6c36df8.jpg,1.0
1,17b1ea55ec3ff6e816908812243319352_original.jpg,c3a5ae8e7.jpg,1.0
2,2b51e87d08e2896e16841412525377045_original.jpg,c2cdf7a78.jpg,1.0
3,60054103c670db9b1677561555761117_original.jpg,cb1a40a3a.jpg,1.0
